## Setup: Create an In-Memory Database

In [1]:
import sqlite3
import pandas as pd

# Create a database connection
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [2]:
# Create tables for Student Management System
cursor.execute('''
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER
);
''')

cursor.execute('''
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT
);
''')

cursor.execute('''
CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(student_id),
    FOREIGN KEY(course_id) REFERENCES courses(course_id)
);
''')
conn.commit()

In [3]:

# INSERT STUDENT DETAILS
Student_data = [
    (101, "RAHUL", 24),
    (102, "SIMRAN", 23),
    (103, "AARAV", 22),
    (104, "PRIYA", 25)
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", Student_data)

# INSERT COURSE DETAILS
Course_data = [
    (5001, "Machine Learning"),
    (5002, "Database Systems"),
    (5003, "Cyber Security"),
    (5004, "Software Engineering")
]

cursor.executemany("INSERT INTO courses VALUES (?, ?)", Course_data)

# INSERT ENROLLMENT DETAILS (WITHOUT enrollment_id, as it's auto-incremented)
Enrollment_data = [
    (101, 5001),  # RAHUL -> Machine Learning
    (102, 5002),  # SIMRAN -> Database Systems
    (103, 5003),  # AARAV -> Cyber Security
    (104, 5004),  # PRIYA -> Software Engineering
    (101, 5002),  # RAHUL -> Database Systems
    (102, 5004),  # SIMRAN -> Software Engineering
    (103, 5001)   # AARAV -> Machine Learning
]

cursor.executemany("INSERT INTO enrollments (student_id, course_id) VALUES (?, ?)", Enrollment_data)

# Commit changes
conn.commit()


In [4]:
# Verification of students table
df = pd.read_sql_query("SELECT * FROM students", conn)
df

,student_id,name,age
0,101,RAHUL,24
1,102,SIMRAN,23
2,103,AARAV,22
3,104,PRIYA,25


In [5]:
# Verification of courses table
df = pd.read_sql_query("SELECT * FROM courses", conn)
df



,course_id,course_name
0,5001,Machine Learning
1,5002,Database Systems
2,5003,Cyber Security
3,5004,Software Engineering


In [6]:
# Verification of enrollments table (with auto-generated enrollment_id)
df = pd.read_sql_query("SELECT * FROM enrollments", conn)
df

,enrollment_id,student_id,course_id
0,1,101,5001
1,2,102,5002
2,3,103,5003
3,4,104,5004
4,5,101,5002
5,6,102,5004
6,7,103,5001


## Importing CSV Data into SQL

In [7]:
# SQL command to import CSV data (example for MySQL/PostgreSQL)
"""
LOAD DATA INFILE 'customers.csv'
INTO TABLE customers
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;
"""

"\nLOAD DATA INFILE 'customers.csv'\nINTO TABLE customers\nFIELDS TERMINATED BY ','\nLINES TERMINATED BY '\n'\nIGNORE 1 ROWS;\n"

In [8]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('customer.csv')  # Replace with your file path

# Import data into SQLite table
df.to_sql('customers', conn, if_exists='replace', index=False)

# Verify data by querying the table
df_from_db = pd.read_sql_query("SELECT * FROM customers", conn)
df_from_db

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


## Fetching API Data and Storing in SQL

In [9]:
# Create the stock_prices table
cursor.execute('''
CREATE TABLE IF NOT EXISTS stock_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    price REAL,
    timestamp TEXT
);
''')
conn.commit()


In [10]:
# API URL
import requests
url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=D2X9JKB7T4N4YK83"
response = requests.get(url)

# Parse the JSON response
data = response.json()

# Extract stock data
stock_data = data.get('Time Series (5min)', {})



# Insert data into the table
for timestamp, stock in stock_data.items():
    cursor.execute("INSERT INTO stock_prices (symbol, price, timestamp) VALUES (?, ?, ?)",
                   ("IBM", stock['4. close'], timestamp))

# Commit the changes
conn.commit()

# Display the table
df = pd.read_sql_query("SELECT * FROM stock_prices", conn)
df

,id,symbol,price,timestamp
0,1,IBM,255.10,2025-01-31 19:55:00
1,2,IBM,255.35,2025-01-31 19:50:00
2,3,IBM,255.43,2025-01-31 19:45:00
3,4,IBM,255.62,2025-01-31 19:40:00
4,5,IBM,255.60,2025-01-31 19:35:00
...,...,...,...,...
95,96,IBM,255.06,2025-01-31 12:00:00
96,97,IBM,254.93,2025-01-31 11:55:00
97,98,IBM,254.97,2025-01-31 11:50:00
98,99,IBM,254.92,2025-01-31 11:45:00


## Filtering Data & Aggregations

In [11]:
# Create the employees table
cursor.execute('''
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary REAL
);
''')
conn.commit()

# Insert sample data into the employees table with some NULL salaries
employee_data = [
    (1, 'Alice', 'Finance', 85000),
    (2, 'Bob', 'Finance', 90000),
    (3, 'Charlie', 'IT', 78000),
    (4, 'David', 'Finance', 95000),
    (5, 'Eve', 'HR', 60000),
    (6, 'Frank', 'Finance', 105000),
    (7, 'Grace', 'IT', 70000),
    (8, 'Hannah', 'Finance', 88000),
    (9, 'Ivy', 'Sales', 70000),
    (10, 'Jack', 'Finance', 110000),
    (11, 'Karen', 'Sales', 75000),
    (12, 'Leo', 'Finance', 97000),
    (13, 'Mona', 'IT', 80000),
    (14, 'Nina', 'Finance', 96000),
    (15, 'Oscar', 'HR', 64000),
    (16, 'Kishan', 'IT', None),  # NULL salary
    (17, 'John', 'IT', None),    # NULL salary
    (18, 'Sophia', 'HR', None)   # NULL salary
]

# Insert data into the employees table
cursor.executemany("INSERT INTO employees (employee_id, name, department, salary) VALUES (?, ?, ?, ?)", employee_data)
conn.commit()


In [12]:
# Retrieve top 5 highest-paid employees in Finance
df_top_finance = pd.read_sql_query(
    "SELECT * FROM employees WHERE department = 'Finance' ORDER BY salary DESC LIMIT 5;",
    conn)
df_top_finance

,employee_id,name,department,salary
0,10,Jack,Finance,110000.0
1,6,Frank,Finance,105000.0
2,12,Leo,Finance,97000.0
3,14,Nina,Finance,96000.0
4,4,David,Finance,95000.0


##  Data Cleaning: Handling NULL Values

In [13]:
# Create a temporary table to store average salaries per department
cursor.execute("""
    CREATE TEMP TABLE dept_avg_salary AS
    SELECT department, AVG(salary) AS avg_salary
    FROM employees
    WHERE salary IS NOT NULL
    GROUP BY department;
""")

# Update NULL salaries using the calculated department averages
cursor.execute("""
    UPDATE employees
    SET salary = (
        SELECT avg_salary 
        FROM dept_avg_salary 
        WHERE dept_avg_salary.department = employees.department
    )
    WHERE salary IS NULL;
""")
conn.commit()

# Check the updated employees table
df = pd.read_sql_query("SELECT * FROM employees", conn)
df


,employee_id,name,department,salary
0,1,Alice,Finance,85000.0
1,2,Bob,Finance,90000.0
2,3,Charlie,IT,78000.0
3,4,David,Finance,95000.0
4,5,Eve,HR,60000.0
5,6,Frank,Finance,105000.0
6,7,Grace,IT,70000.0
7,8,Hannah,Finance,88000.0
8,9,Ivy,Sales,70000.0
9,10,Jack,Finance,110000.0


##  Joins - Combining Data from Multiple Tables

In [14]:
# Step 1: Create the departments table if it does not exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT UNIQUE
);
''')
conn.commit()

# Step 2: Insert department data (ignoring duplicates)
departments_data = [
    (1, "Finance"),
    (2, "IT"),
    (3, "HR"),
    (4, "Marketing")
]
cursor.executemany("INSERT OR IGNORE INTO departments VALUES (?, ?)", departments_data)
conn.commit()

# Step 3: Update employees' department names to department IDs
cursor.execute('''
UPDATE employees
SET department = (SELECT department_id FROM departments WHERE departments.department_name = employees.department)
WHERE department IS NOT NULL;
''')
conn.commit()

# Step 4: Join employees and departments tables
df_joined = pd.read_sql_query(
    """
    SELECT e.name, d.department_name 
    FROM employees e 
    INNER JOIN departments d ON e.department = d.department_id;
    """, conn)

# Display the joined table
df_joined


,name,department_name
0,Alice,Finance
1,Bob,Finance
2,Charlie,IT
3,David,Finance
4,Eve,HR
5,Frank,Finance
6,Grace,IT
7,Hannah,Finance
8,Jack,Finance
9,Leo,Finance


## Subqueries & Nested Queries

In [15]:
# Find employees whose salary is greater than department average
df_above_avg = pd.read_sql_query(
    """
    SELECT name, salary, department 
    FROM employees 
    WHERE salary > (SELECT AVG(salary) FROM employees AS e2 WHERE e2.department = employees.department);
    """, conn)
df_above_avg

,name,salary,department
0,Charlie,78000.0,2
1,Frank,105000.0,1
2,Jack,110000.0,1
3,Leo,97000.0,1
4,Mona,80000.0,2
5,Nina,96000.0,1
6,Oscar,64000.0,3


## Window Functions - Ranking & Running Totals

In [16]:
# Rank employees based on salary
df_ranked = pd.read_sql_query(
    """
    SELECT name, salary, RANK() OVER (ORDER BY salary DESC) AS salary_rank 
    FROM employees;
    """, conn)
df_ranked

,name,salary,salary_rank
0,Jack,110000.0,1
1,Frank,105000.0,2
2,Leo,97000.0,3
3,Nina,96000.0,4
4,David,95000.0,5
5,Bob,90000.0,6
6,Hannah,88000.0,7
7,Alice,85000.0,8
8,Mona,80000.0,9
9,Charlie,78000.0,10


## Optimizing Query Performance

In [17]:
# Creating an index for faster search
cursor.execute("CREATE INDEX idx_salary ON employees(salary);")
conn.commit()

## Handling JSON Data in SQL

In [18]:
# Create the 'purchases' table with a JSON column
cursor.execute('''
CREATE TABLE purchases (
    purchase_id INTEGER PRIMARY KEY,
    json_column TEXT
);
''')
conn.commit()

# Insert JSON data into the 'purchases' table
purchases_data = [
    (1, '{"customer_name": "Alice", "purchase_amount": 250}'),
    (2, '{"customer_name": "Bob", "purchase_amount": 450}'),
    (3, '{"customer_name": "Charlie", "purchase_amount": 320}')
]

cursor.executemany('INSERT INTO purchases (purchase_id, json_column) VALUES (?, ?)', purchases_data)
conn.commit()


In [19]:
# Convert JSON fields into SQL columns (Example for MySQL)
cursor.execute("""
SELECT json_extract(json_column, '$.customer_name') AS customer_name,
       json_extract(json_column, '$.purchase_amount') AS purchase_amount
FROM purchases;
""")

# Fetch and display the result
result = cursor.fetchall()
for row in result:
    print(row)

('Alice', 250)
('Bob', 450)
('Charlie', 320)


## Closing the Database Connection

In [20]:
conn.close()